In [1]:
import pandas as pd
import numpy as np
import torch
import h5py
from torchvision.transforms import (
    Compose,
    RandomHorizontalFlip,
    RandomVerticalFlip,
)

/home/zeerak.talat/.conda/envs/featuregen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
idx = 6979

In [3]:
df = pd.read_pickle("data/more_text_IEMO.pkl")
AUDIOS = (h5py.File("data/whisper_iemo_audio.hdf5", "r", libver="latest", swmr=True))
VIDEOS = (h5py.File("data/iemo_videos_blackground.hdf5", "r", libver="latest", swmr=True))

# Checking Size

In [4]:
len(list(AUDIOS.keys())), len(list(VIDEOS.keys())), len(df)

(7380, 7380, 7380)

In [5]:
def split_len(split):    
    i = 0
    for key in AUDIOS.keys():
        if key.startswith(split):
            i += 1
    return i

def df_len(split):
    return len(df[df["split"] == split])

split_len("train"), split_len("val"), split_len("test")

(4433, 1472, 1475)

In [6]:
df_len("train"), df_len("val"), df_len("test")

(4778, 980, 1622)

# Code

In [7]:
df.iloc[idx]

text               LINDA: You kept saying my flashlight as if it'...
index                                              Ses04M_script02_1
split                                                            val
emotion                                                            4
utterance_order                                                   23
talker                                                       MICHAEL
sentiment_label                                              excited
dialog                                                           116
sentiment                                                          4
timings                                            (135.3091, 149.4)
bbox               [[[64, 48, 165, 216]], [[63, 46, 166, 218]], [...
audio_path              ../../data/iemocap_wav/Ses04M_script02_1.wav
video_path              ../../data/iemocap_avi/Ses04M_script02_1.avi
len                                                               16
emotion_label                     

In [18]:
df.iloc[idx]['bbox']

'[[[58, 17, 186, 219]], [[59, 16, 185, 218]], [[59, 16, 185, 217]], [[59, 17, 185, 219]], [[58, 17, 186, 219]], [[58, 16, 185, 218]], [[59, 16, 185, 217]], [[58, 16, 186, 219]], [[59, 17, 185, 219]], [[59, 16, 185, 220]], [[53, 16, 182, 220]], [[60, 15, 184, 220]], [[60, 15, 184, 219]], [[59, 15, 185, 219]], [[58, 14, 185, 217]], [[59, 15, 185, 218]]]'

In [8]:
audio_path = df.iloc[idx]['audio_path']
video_path = df.iloc[idx]['video_path']
timings = df.iloc[idx]['timings']

## Audio

In [11]:
speech_array = torch.Tensor(AUDIOS[f"val_{audio_path.split('/')[-1][:-4]}_{timings}"][()])

In [12]:
torch.save(speech_array, f"{idx}_audio.pt")

## Video

In [14]:
transform = Compose(
    [
        RandomHorizontalFlip(p=0),  #
        RandomVerticalFlip(p=0),  #
    ]
)
video = torch.Tensor(VIDEOS[f"val_{video_path.split('/')[-1][:-4]}_{timings}"][()])  
video = transform(video)


In [15]:
torch.save(video, f"{idx}_video.pt")